In [1]:
import os
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt

from utils.dataloader import DataLoader
from utils.vis import MatplotlibVisualizer
from utils.transforms import HairRemoval, Composer
from descriptors.shape import HOGDescriptor
from descriptors.texture import LBPDescriptor, GLCMDescriptor
from utils.segmentation import ThresholdingSegmentation
from descriptors.color import ColorDescriptor, ColorLayoutDescriptor, ColorCooccurrenceMatrixDescriptor

In [2]:
## Classes
CLASSES = ['nevus', 'others']

## Work folfer
work_folder = os.getcwd()
data_folder = os.path.join(work_folder, '..', 'Data/Challenge1')

## Visualizer
matplotlib_visualizer = MatplotlibVisualizer()

In [3]:
transforms_composer = Composer([
                                HairRemoval(),
                                ])
otsu_thresholding = ThresholdingSegmentation(method='otsu')

## Featrure Extraction

### Color Descriptor

In [ ]:
modes = ['train', 'val']
## Descriptors
color_descriptor = ColorDescriptor(bins=(8, 12, 3))
color_cooccurrence_matrix_descriptor = ColorCooccurrenceMatrixDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8)
# glcm_descriptor = GLCMDescriptor(distances=[1, 2, 3], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8)
lbp_descriptor = LBPDescriptor(radius=1, n_points=8, grid_x=5, grid_y=5)

for mode in modes:
    ## Make a new folder for the features
    os.makedirs(os.path.join('feautres', mode, 'color'), exist_ok=True)

    ## Data loader
    ### Limit the number of samples to 200 for training and load all samples for validation
    max_samples = None if mode == 'train' else None
    ### Balance the dataset for training
    balance = False if mode == 'train' else False
    dataloader = DataLoader(data_folder, mode, 
                            shuffle=True, 
                            ignore_folders=['black_background', '.DS_Store'], 
                            max_samples=max_samples, 
                            balance=balance,
                            transforms=None)

    ## Extract features
    features = []
    labels = []
    for i, (img, label, path) in tqdm(enumerate(dataloader), total=len(dataloader), desc=f'Extracting features for {mode}'):
        ## combine all descriptors
        # mask = otsu_thresholding(img)
        # mask = cv2.bitwise_not(mask)
        mask = None
        color_features = color_descriptor.extract(img, mask=mask)
        color_cooccurrence_matrix_features = color_cooccurrence_matrix_descriptor.extract(img, mask=mask)
        # glcm_features = glcm_descriptor.extract(img, mask=mask)
        lbp_features = lbp_descriptor.extract(img, mask=mask)
        # hog_features = hog_descriptor.extract(img)
        features.append(np.concatenate([color_features,color_cooccurrence_matrix_features,lbp_features]))
        
        
        ## add label
        labels.append(label)

    ## Save features to disk
    features = np.array(features)
    labels = np.array(labels)
    features_with_labels = np.concatenate([features, labels.reshape(-1, 1)], axis=1)
    np.save(os.path.join('feautres', mode, 'color', 'features.npy'), features_with_labels)

## Training

In [18]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb

In [27]:
mode = 'train'
# model = SVC(kernel='rbf', C=5.0, random_state=42, degree=5)
model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=150, learning_rate=0.05, n_jobs=-1)


In [28]:
features_with_labels = np.load(os.path.join('feautres', mode, 'color', 'features.npy'))
features = features_with_labels[:, :-1]
labels = features_with_labels[:, -1]

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

## Validation

In [29]:
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.75      0.74      0.74       860
      others       0.83      0.83      0.83      1292

    accuracy                           0.79      2152
   macro avg       0.79      0.78      0.79      2152
weighted avg       0.79      0.79      0.79      2152



## Testing

In [30]:
# mode = 'val'
features_with_labels_test = np.load(os.path.join('feautres', 'val', 'color', 'features.npy'))
features_test = features_with_labels_test[:, :-1]
labels_test = features_with_labels_test[:, -1]

y_pred = model.predict(features_test)
print(classification_report(labels_test, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.76      0.73      0.75      1052
      others       0.83      0.85      0.84      1583

    accuracy                           0.80      2635
   macro avg       0.80      0.79      0.79      2635
weighted avg       0.80      0.80      0.80      2635

